In [ ]:
#!pip install nisanyan-cli

In [ ]:
#!pip uninstall nisanyan-cli -y

In [ ]:
#!pip install scrapy

In [ ]:
#!pip install orjson

In [ ]:
#!pip install --upgrade pip

In [1]:
import scrapy
import pandas as pd
import numpy as np
import glob
import re
import nltk
from nltk import word_tokenize
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
from urllib.parse import quote
from urllib.request import Request
from urllib import parse
import orjson

In [8]:
# -*- coding: utf-8 -*-
# import argparse

#import requests
#from requests.adapters import HTTPAdapter
#from urllib3.util import Retry
#from urllib.parse import quote
#
#from bs4 import BeautifulSoup
#import orjson

# data dictionary
data_dict = {}

# Given words
first_word = "ab"
last_word = "zürriyet"

# Filenames
f_output = "output.json"
f_wordlist = "wordlist.txt"

# Request
def req(word):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    url = "https://www.nisanyansozluk.com/?k=" + quote(word) + "&lnk=1&view=annotated"
    session.mount(url, adapter)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
        "Accept-Encoding": "*",
        "Connection": "keep-alive",
        "accept-encoding": "gzip, deflate, br",
        "cache-control": "max-age=0",
        "content-type": "application/x-www-form-urlencoded",
        "dnt": "1",
        "upgrade-insecure-requests": "1",
    }
    r = session.get(url, headers=headers)
    # print(session.cookies.get_dict())
    return r.content


# Sometimes distortions occur when going sequentially in the table. e.g.: add -> "adem"
def back_forward():
    try:
        soup = BeautifulSoup(req(list(data_dict)[-2]), "html5lib")
        cell = soup.find("tr", {"class": "yaz hghlght"})
        nextt_word = cell.nextSibling.nextSibling.nextSibling.nextSibling.td["title"]

        print(
            "back_forward: "
            + list(data_dict)[-2]
            + " => \033[95m"
            + list(data_dict)[-1]
            + "\033[0m => "
            + nextt_word
        )
    except AttributeError:
        soup = BeautifulSoup(req(list(data_dict)[-3]), "html5lib")
        cell = soup.find("tr", {"class": "yaz hghlght"})
        nextt_word = cell.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.td[
            "title"
        ]
        print(
            "back_back_forward: "
            + list(data_dict)[-3]
            + " => \033[95m"
            + list(data_dict)[-2]
            + ", "
            + list(data_dict)[-1]
            + "\033[0m => "
            + nextt_word
        )
    gg(nextt_word)


# final message
def final_msg(msg):
    p_green = "\033[92m"
    p_blue = "\033[94m"
    p_endc = "\033[0m"

    print("-------------------------\n")
    print(p_blue + msg + p_endc + "\n")
    print(
        p_blue
        + "İlk kelime: "
        + p_green
        + list(data_dict)[0]
        + p_blue
        + ", Son kelime: "
        + p_green
        + list(data_dict)[-1]
    )
    print(p_blue + "Toplam kelime sayısı: " + p_green + str(len(data_dict)) + p_endc)
    print(p_blue + "Çıktı dosyası: " + p_green + f_output + p_endc)
    print(p_blue + "Kelime listesi dosyası: " + p_green + f_wordlist + p_endc)
    print("\a")


def export():
    # export data to .json file
    with open(f_output, "wb") as f:
        f.write(orjson.dumps(data_dict))

    # export wordlist to .txt file
    with open(f_wordlist, "w", encoding="utf-8") as f:
        f.write("\n".join(list(data_dict)))


# scrape the cell
def scrape_cell(cell):
    cell = cell
    kelime = cell.td["title"]
    tarih = cell.find("div", {"class": "maddetarih"}).text
    eskoken = cell.find_all("div", {"class": "eskoken"})
    baslik = (
        tarihce
    ) = koken = daha_fazla = ek_aciklama = benzer_sozcukler = maddeye_gonderenler = ""
    baslik = cell.td.a.text

    for i in eskoken:
        if i.div != None:
            title = i.div.text
            if "Tarihçe" in title:
                tarihce = str(i.p)
            elif "Köken" in title:
                koken = str(i.p)
            elif "Ek açıklama" in title:
                ek_aciklama = str(i.p)
            elif "Benzer sözcükler" in title:
                benzer_sozcukler = list(i.p.text.split(", "))
            elif "Bu maddeye gönderenler" in title:
                maddeye_gonderenler = list(i.p.text.strip().split(", "))
        elif "Daha fazla bilgi" in i.p.text:
            k = []
            a = i.p.find_all("a")
            for i in a:
                k.append(i.text)
            daha_fazla = k

    data = {
        "baslik": baslik,
        "tarihce": tarihce,
        "koken": koken,
        "daha_fazla": daha_fazla,
        "ek_aciklama": ek_aciklama,
        "benzer_sozcukler": benzer_sozcukler,
        "maddeye_gonderenler": maddeye_gonderenler,
        "tarih": tarih,
    }
    data_dict[kelime] = data
    print(kelime)


# Main function gg
def gg(first, last):
    word = first
    lastw = last
    print(word + " ➔ " + lastw + "\n")

    done = False

    while not done:
        content = req(word)
        soup = BeautifulSoup(content, "html.parser")
        cell = soup.find("tr", {"class": "yaz hghlght"})
        all_cells = soup.find_all("tr", {"class": "yaz hghlght"})

        # Wrong word
        if len(all_cells) == 0:
            print("Girilen kelime bulunamadı!")
            break
        # Double word e.g.: ram,RAM
        elif len(all_cells) > 1:
            for i in all_cells:
                scrape_cell(i)
        else:
            scrape_cell(cell)

        kelime = cell.td["title"]
        if kelime == lastw:
            done = True
        else:

            if (
                cell.nextSibling.nextSibling != None
                and cell.nextSibling.nextSibling.nextSibling.nextSibling != None
            ):
                if (
                    cell.nextSibling.nextSibling.find("div", {"class": "etymtxt"}).text
                    == ""
                ):
                    word = cell.nextSibling.nextSibling.nextSibling.nextSibling.td[
                        "title"
                    ]
                    continue
                word = cell.nextSibling.nextSibling.td["title"]
            else:
                back_forward()
    else:
        export()
        final_msg("Başarıyla tamamlandı...")
        exit()


def main(first=first_word, last=last_word):
    try:
        gg(first, last)
    except KeyboardInterrupt:
        export()
        final_msg("Liste tamamlanamadı!\nKeyboardInterrupt")
    except Exception as e:
        export()
        final_msg("Liste tamamlanamadı!\nBeklenmedik bir durum oluştu: " + str(e))

In [9]:
main()

ab ➔ zürriyet



Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Girilen kelime bulunamadı!


In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
from urllib.parse import quote

from bs4 import BeautifulSoup
import orjson

In [ ]:
def req(word):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    url = "https://www.nisanyansozluk.com/?k=" + quote(word) + "&lnk=1&view=annotated"
    session.mount(url, adapter)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
        "Accept-Encoding": "*",
        "Connection": "keep-alive",
        "accept-encoding": "gzip, deflate, br",
        "cache-control": "max-age=0",
        "content-type": "application/x-www-form-urlencoded",
        "dnt": "1",
        "upgrade-insecure-requests": "1",
    }
    r = session.get(url, headers=headers)
    # print(session.cookies.get_dict())
    return r.content

In [ ]:
req("elma")

#### Arabic Language Process

In [ ]:
df_clean = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_cross_check2.csv")
#df_clean = df_clean.head(150000)
df_clean

In [ ]:
#df_test = df_clean[df_clean.lang == "Arabic"]
#df_test

In [ ]:
#df_test["term"].nunique()

In [ ]:
df_related_arabic = df_clean[df_clean.related_lang == "Arabic"]
df_related_arabic

In [ ]:
df_select_lang = df_related_arabic[df_related_arabic.lang == "Turkish"]
df_select_lang

In [ ]:
df_var1 = pd.DataFrame(df_select_lang["term"].unique(), columns=["term"])
df_var1

In [ ]:
df_var1.to_excel("Turkish_Term.xlsx", index=False)

In [ ]:
df_var2 = pd.DataFrame(df_select_lang["related_term"].unique(), columns=["related_term"])
df_var2

In [ ]:
df_var2.to_excel("Arabic_Related_Term.xlsx", index=False)

In [ ]:
df_var1_translate = pd.read_excel("Turkish Arabic Language Pair Translate.xlsx")
df_var1_translate

In [ ]:
df_var2.rename(columns={"related_term":"arabic_word"}, inplace=True)
df_var2

In [ ]:
pd.merge(df_var2,df_var1_translate, how="inner",on="arabic_word")

In [ ]:
df_var2_translate = pd.read_excel("Arabic Turkish Language Pair Translate.xlsx")
df_var2_translate

In [ ]:
df_var1.rename(columns={"term":"turkish_word"}, inplace=True)
df_var1

In [ ]:
pd.merge(df_var1,df_var2_translate, how="inner",on="turkish_word")

#### Etimoloji Türkçe

In [1]:
df_ety_sozluk = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/Turkish_0-40000_Word_Etimoloji_Result.xlsx")
df_ety_sozluk

<IPython.core.display.Javascript object>

,search_word,word,lang,koken,lang_root
0,bir,bir,Eski Türkçe,Eski Türkçe bir sözcüğünden evrilmiştir.,Eski Türkçe
1,ve,ve,Arapça,"Arapça wa و z ""ve (bağlaç), - adına (edat)"" fi...",Arapça
2,için,i̇çin,Eski Türkçe,"Eski Türkçe üçün ""nedensellik edatı"" sözcüğünd...",Eski Türkçe
3,çok,çok,Ana Türkçe,"Ana Türkçe yazılı örneği bulunmayan *çaw- ""bağ...",Ana Türkçe
4,ama,ama,Arapça,"Arapça ammā أمّا z ""gelgelelim, maamafih (bağl...",Arapça
...,...,...,...,...,...
3701,masör,masör,Fransızca,"Fransızca masseur ""masaj yapan"" sözcüğünden al...",Fransızca
3702,baraka,baraka,İtalyanca,"İtalyanca baracca ""kulübe, derme çatma yapı"" s...",İtalyanca
3703,oğlak,oğlak,Eski Türkçe,Eski Türkçe ogul sözcüğünden +Ak sonekiyle tür...,Eski Türkçe
3704,boykot,boykot,İngilizce,"İngilizce boycott ""bir mal veya hizmeti satın ...",İngilizce


In [4]:
df_ety_sozluk[(df_ety_sozluk["lang"] == "Türkçe")]

,search_word,word,lang,koken,lang_root
19,sonra,sonra,Türkçe,Türkçe soŋ sözcüğünden +rA sonekiyle türetilmi...,Türkçe
31,başka,başka,Türkçe,Türkçe baş sözcüğünden +kA sonekiyle türetilmi...,Türkçe
37,hey,hey,Türkçe,Türkçe ünlemdir.,ünlemdir.
50,sorun,sorun,Türkçe,Türkçe sor- fiilinden +In sonekiyle türetilmiş...,Türkçe
61,haydi,haydi,Türkçe,Türkçe hay de! deyiminden evrilmiştir. Türkçe ...,Türkçe
...,...,...,...,...,...
3584,çiti,çiti,Türkçe,"Türkçe çit- ""elde çamaşır ovalamak"" fiilinden ...",Türkçe
3607,nere,nere,Türkçe,"Türkçe ne ""soru sıfatı ve zamiri"" sözcüğünden ...",Türkçe
3657,çekince,çekince,Türkçe,Türkçe çekin- fiilinden +çA sonekiyle türetilm...,Türkçe
3662,algı,algı,Türkçe,Türkçe al- fiilinden +gU sonekiyle türetilmişt...,Türkçe


In [6]:
df_ety_sozluk[(df_ety_sozluk["lang"] == "Türkçe") & (df_ety_sozluk["lang_root"] == "Türkçe")]

,search_word,word,lang,koken,lang_root
19,sonra,sonra,Türkçe,Türkçe soŋ sözcüğünden +rA sonekiyle türetilmi...,Türkçe
31,başka,başka,Türkçe,Türkçe baş sözcüğünden +kA sonekiyle türetilmi...,Türkçe
50,sorun,sorun,Türkçe,Türkçe sor- fiilinden +In sonekiyle türetilmiş...,Türkçe
61,haydi,haydi,Türkçe,Türkçe hay de! deyiminden evrilmiştir. Türkçe ...,Türkçe
64,kimse,kimse,Türkçe,Türkçe kim ise deyiminden türetilmiştir. Daha ...,Türkçe
...,...,...,...,...,...
3584,çiti,çiti,Türkçe,"Türkçe çit- ""elde çamaşır ovalamak"" fiilinden ...",Türkçe
3607,nere,nere,Türkçe,"Türkçe ne ""soru sıfatı ve zamiri"" sözcüğünden ...",Türkçe
3657,çekince,çekince,Türkçe,Türkçe çekin- fiilinden +çA sonekiyle türetilm...,Türkçe
3662,algı,algı,Türkçe,Türkçe al- fiilinden +gU sonekiyle türetilmişt...,Türkçe


#### Nişanyan

In [13]:
df_nisan = pd.read_excel("Nisanyan Words For English.xlsx")
df_nisan

,term,related_lang,related_term
0,+ark,Fransızca\n,+arque
1,+arşi,Fransızca\n,+archie
2,+faj,Fransızca\n ve İngilizce\n,+phage
3,+gam,Fransızca\n,+game
4,+gen,Fransızca\n,+gone
...,...,...,...
6012,zula,Yunanca,zúla
6013,zum,İngilizce\n,zoom
6014,zurna,eng,zurna
6015,اوطه‌لق,fra,odalisque


In [70]:
df_nisan["term"] = df_nisan["term"].apply(lambda x: x.strip("\+"))
df_nisan["related_term"] = df_nisan["related_term"].apply(lambda x: str(x).strip("\+"))
df_nisan

,term,related_lang,related_term
0,ark,Fransızca\n,arque
1,arşi,Fransızca\n,archie
2,faj,Fransızca\n ve İngilizce\n,phage
3,gam,Fransızca\n,game
4,gen,Fransızca\n,gone
...,...,...,...
6012,zula,Yunanca,zúla
6013,zum,İngilizce\n,zoom
6014,zurna,eng,zurna
6015,اوطه‌لق,fra,odalisque


In [5]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = df_word.head(10000)
df_word

,word,frequency
0,bir,18835735
1,fikret,12101580
2,bu,11062659
3,ne,8025880
4,ve,7766036
...,...,...
164296,buldugumuzda,43
164297,boşandıklarını,42
164298,endişenlenme,42
164299,uzaylıymışım,42


In [6]:
set_nisanyan = set(df_nisan.term)
set_word = set(df_word.word)

In [10]:
df_nisan_word_intersect = pd.DataFrame(set_nisanyan.intersection(set_word), columns=["term"])
df_nisan_word_intersect

,term
0,roof
1,keton
2,kontrat
3,vantuz
4,gondol
...,...
2746,aort
2747,kukuleta
2748,lejyon
2749,hardcore


In [12]:
df_nisan_word_merge = pd.merge(df_nisan,df_nisan_word_intersect,how="inner",on="term")
df_nisan_word_merge

,term,related_lang,related_term
0,ark,Fransızca\n,ark
1,gam,Fransızca\n,gam
2,gen,Fransızca\n,gen
3,gen,fra,gen
4,gon,Fransızca\n,gon
...,...,...,...
2752,zooloji,Fransızca\n,zooloji
2753,zoraki,Yunanca,zoraki
2754,zula,Yunanca,zula
2755,zum,İngilizce\n,zum


### Turkish English Works

In [4]:
lang_1 = "Turkish"  # English, Turkish, German, French, Spanish, Arabic, Russian, Portuguese, Dutch, Italian 
lang_2 = "English"  # English, Turkish, German, French, Spanish, Arabic, Russian, Portuguese, Dutch, Italian

if lang_2 == "English":
    lang_3 = "English (eng)"
elif lang_2 == "Turkish":
    lang_3 = "Turkish (tur)"
elif lang_2 == "German":
    lang_3 = "German (deu)"
elif lang_2 == "French":
    lang_3 = "French (fra)"
elif lang_2 == "Spanish":
    lang_3 = "Spanish (spa)"
elif lang_2 == "Arabic":
    lang_3 = "Arabic (ara)"
elif lang_2 == "Russian":
    lang_3 = "Russian (rus)"
elif lang_2 == "Portuguese":
    lang_3 = "Portuguese (por)"
elif lang_2 == "Dutch":
    lang_3 = "Dutch (nld)"
elif lang_2 == "Italian":
    lang_3 = "Italian (ita)"
else:
    pass

print(f"{lang_1}\n{lang_2}\n{lang_3}")

Turkish
English
English (eng)


In [3]:
df_all_cross = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_1.capitalize()}/{lang_1.capitalize()}_All_Word_From_{lang_2.capitalize()}_Etymology.xlsx")
df_all_cross = df_all_cross.loc[:,["term_master","lang_sleeve","term_sleeve"]]
df_all_cross

,term_master,lang_sleeve,term_sleeve
0,sirk,English,cirque
1,reis,English,reis
2,espri,English,esprit
3,oksijen,English,oxygen
4,kaza,English,caza
...,...,...,...
93801,regülasyon,English,regulation
93802,levitasyon,English,gravitate
93803,haber spikeri,English,speaker
93804,radyo spikeri,English,radio


In [4]:
df_all_cross["term_master"].nunique()

2289

In [5]:
df_ety_turk = pd.read_excel("Etimolojiturk.xlsx")
df_ety_turk.rename(columns={"word":"term_master","lang_root":"lang_sleeve","koken":"term_sleeve"}, inplace=True)
df_ety_turk = df_ety_turk.loc[:,["term_master","lang_sleeve","term_sleeve"]]
df_ety_turk

,term_master,lang_sleeve,term_sleeve
0,kontrol,Latince,"Fransızca contrôle ""denetim"" sözcüğünden alınt..."
1,doktor,Latince,"Fransızca docteur ""1. üniversitede hocalık ber..."
2,polis,Eski Yunanca,"Fransızca police ""1. kamu düzeni [esk.], 2. po..."
3,dolar,Almanca,İngilizce aynı anlama gelen dollar sözcüğünden...
4,alo,Eski Fransızca,"Fransızca allo ""telefon hitabı"" sözcüğünden al..."
...,...,...,...
1180,şike,Fransızca,"Fransızca chiqué ""dalavereli"" sözcüğünden alın..."
1181,eşofman,Fransızca,"Fransızca échauffement ""ısınma, ısıtma, sporda..."
1182,masör,Fransızca,"Fransızca masseur ""masaj yapan"" sözcüğünden al..."
1183,baraka,İtalyanca,"İtalyanca baracca ""kulübe, derme çatma yapı"" s..."


In [6]:
df_dont_plus = pd.read_excel("Nisanyan Words Dont Contain Plus.xlsx")
df_dont_plus.rename(columns={"term":"term_master","related_lang":"lang_sleeve","related_term":"term_sleeve"}, inplace=True)
df_dont_plus

,term_master,lang_sleeve,term_sleeve
0,abajur,fra,abat-jour
1,abaküs,fra,abacus
2,abandone,Fransızca,s'abandonner
3,abdest,eng,abdest
4,abdomen,Latince,"abdomen, abdomin-"
...,...,...,...
5857,zula,Yunanca,zúla
5858,zum,İngilizce,zoom
5859,zurna,eng,zurna
5860,اوطه‌لق,fra,odalisque


In [7]:
df_github_ety_nisan_concat = pd.concat([df_all_cross,df_ety_turk,df_dont_plus], axis=0)
df_github_ety_nisan_concat.drop_duplicates(inplace=True)
df_github_ety_nisan_concat

,term_master,lang_sleeve,term_sleeve
0,sirk,English,cirque
1,reis,English,reis
2,espri,English,esprit
3,oksijen,English,oxygen
4,kaza,English,caza
...,...,...,...
5857,zula,Yunanca,zúla
5858,zum,İngilizce,zoom
5859,zurna,eng,zurna
5860,اوطه‌لق,fra,odalisque


In [8]:
df_github_ety_nisan_concat.term_master.nunique()

7171

In [10]:
df_github_ety_nisan_concat.to_excel("Turkish_English_Github_Etimoloji_Nisanyan_Concat_Result.xlsx", index=False)

In [63]:
set_df_cross = set(df_all_cross["term_master"])
set_df_ety_turk = set(df_ety_turk["term_master"])
set_df_nisan = set(df_dont_plus["term_master"])

In [64]:
df_turkish_english = pd.DataFrame((set_df_cross.union(set_df_ety_turk)).union(set_df_nisan), columns=["word_unique"])
df_turkish_english["word_unique"] = df_turkish_english["word_unique"].apply(lambda x: x.lower())
df_turkish_english

,word_unique
0,solvent
1,dvd
2,londralı
3,koli2
4,aferist
...,...
7166,monokl
7167,zero
7168,tatlı
7169,neon


In [54]:
df_turkish_english.to_excel("Turkish_English_All_Unique_Word.xlsx", index=False)

##### All Unique Word Github Intersect

In [24]:
df_github_cross = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_cross_check2.csv")
df_github_cross

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,inherited_from,9w90qvZDUH-2OHPVZEGJSQ,Middle english,parlai
1,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,Uy6v5wWuW6KGH4aTqGWnmA,Old french,parler
2,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,HY24XyCcUZCYSa1Q6y2kPA,Late latin,parabolō
3,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,uXGtk5g-W8G_TlTxSHImpg,Latin,parabola
4,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,JHVmHqAPXEq-YaIbd0B5Jg,Ancient greek,παραβολή
...,...,...,...,...,...,...,...
8854119,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,inherited_from,TUnsO83vXXC1Us5rJgGyUg,Vulgar latin,*pittitus
8854120,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,oUnwLLsHVymmwwiRXgrRow,Latin,pitinnus
8854121,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,IGRZaHQ-VU-vpxMjdxSTlA,Latin,pitulus
8854122,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,MDRrWciZWXGEj1KR9MUWtg,Latin,piccinus


In [25]:
df_github_turkish = df_github_cross[df_github_cross["lang"] == "Turkish"]
df_github_turkish

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
110,caAhC_cAXrKZU9rCcPAaPw,Turkish,göl,inherited_from,T6Wxr8YPVWK47B34DsG6KA,Proto-turkic,*kȫl
817,SC0fKM3-VcGTWvelEkpEXw,Turkish,ön,inherited_from,nlVlgr5dUjas0_4GqplLJQ,Proto-turkic,*ǖn
818,SC0fKM3-VcGTWvelEkpEXw,Turkish,ön,inherited_from,5g1DdR5OUPOxiQxCVcGW6A,Proto-turkic,*öŋ
819,SC0fKM3-VcGTWvelEkpEXw,Turkish,ön,inherited_from,6KC332NQU7Sx9sFKEv6QdQ,Proto-turkic,*oŋ
1345,5bVUD53NVEWYY0QAetV7iA,Turkish,net,inherited_from,Ru1rzGVDWpi2rTYD9-XJWQ,Old french,net
...,...,...,...,...,...,...,...
8853155,nxRVOzFvUzS4X7r165GWcw,Turkish,ordinatör,etymologically_related_to,tPEd3hNNXqeBt17YkY3I2Q,French,calculateur
8853651,pl-cU0pBVda2_BuEjBmtZw,Turkish,virman,has_prefix_with_root,9CJvE6k0WJmKvcWZQmQjxw,French,virer
8853652,pl-cU0pBVda2_BuEjBmtZw,Turkish,virman,has_suffix,8YtrMi5VXlSpZVOupUTcWg,French,ment
8853831,fc8biERQXFicQGtfESBJQQ,Turkish,angajman,has_prefix_with_root,qQY-6sksXJe832mC9wIBqQ,French,engager


In [27]:
df_turkish_english = pd.read_excel("Turkish_English_All_Unique_Word.xlsx")
df_turkish_english

,word_unique
0,solvent
1,dvd
2,londralı
3,koli2
4,aferist
...,...
7166,monokl
7167,zero
7168,tatlı
7169,neon


In [28]:
set_github_turkish = set(df_github_turkish["term"])
set_all_unique = set(df_turkish_english["word_unique"])

In [29]:
df_all_unique_word_github = pd.DataFrame(set_github_turkish.intersection(set_all_unique), columns=["word_intersect"])
df_all_unique_word_github

,word_intersect
0,altın sinek
1,terim
2,kontrat
3,gondol
4,sör
...,...
2319,plaj
2320,tabure
2321,peyzaj
2322,tatlı


##### All Unique Word All Data Intersect

In [11]:
df_github_ety_nisan_concat = pd.read_excel("Turkish_English_Github_Etimoloji_Nisanyan_Concat_Result.xlsx")
df_github_ety_nisan_concat

,term_master,lang_sleeve,term_sleeve
0,sirk,English,cirque
1,reis,English,reis
2,espri,English,esprit
3,oksijen,English,oxygen
4,kaza,English,caza
...,...,...,...
100834,zula,Yunanca,zúla
100835,zum,İngilizce,zoom
100836,zurna,eng,zurna
100837,اوطه‌لق,fra,odalisque


In [12]:
df_github_ety_nisan_concat.term_master.nunique()

7171

In [13]:
df_turkish_english = pd.read_excel("Turkish_English_All_Unique_Word.xlsx")
df_turkish_english.rename(columns={"word_unique":"term_master"}, inplace=True)
df_turkish_english

,term_master
0,solvent
1,dvd
2,londralı
3,koli2
4,aferist
...,...
7166,monokl
7167,zero
7168,tatlı
7169,neon


In [16]:
df_merge_all = pd.merge(df_github_ety_nisan_concat,df_turkish_english, how="inner", on="term_master")
df_merge_all.drop_duplicates(inplace=True)
df_merge_all

,term_master,lang_sleeve,term_sleeve
0,sirk,English,cirque
1,sirk,English,crucial
2,sirk,English,cruciform
3,sirk,English,cross
4,sirk,English,incarcerate
...,...,...,...
101086,zoraki,Yunanca,zoráki
101087,zula,Yunanca,zúla
101088,zum,İngilizce,zoom
101089,اوطه‌لق,fra,odalisque


In [43]:
df_merge_all.to_excel("Turkish_English_All_Unique_Word_Related_Lang_And_Term.xlsx", index=False)

In [41]:
df_merge_all["term_master"].nunique()

7134

### Word Analysis

In [17]:
df_turkish_english_all = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_English (eng)_Etymologeek_Result_All.xlsx")
df_turkish_english_all

,dict_entry_main,english_word
0,sirk,circus
1,oksijen,oxygen
2,kaza,accident
3,sinema,cinema
4,sezon,season
...,...,...
1335,çita,cheetah
1336,çubuk,chibouk
1337,şiş,shish
1338,şiş kebap,shishkabob


In [19]:
df_turkish_english_shared = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_English_Shared_Vocabulary.xlsx")
df_turkish_english_shared

,dict_entry_main,english_word
0,abaküs,abacus
1,abandone,abandon
2,abanoz,ebony
3,abdest,ablution
4,abdesthane,ablution room
...,...,...
2132,zodyak,zodiac
2133,zombi,zombie
2134,zooloji,zoology
2135,zoolojik,zoological


In [21]:
set_all = set(df_turkish_english_all["dict_entry_main"])
set_shared = set(df_turkish_english_shared["dict_entry_main"])

In [36]:
df_diff = pd.DataFrame(set_all.difference(set_shared), columns=["dict_entry_main"])
df_diff

,dict_entry_main
0,arayüz
1,abece
2,gey
3,tulumba
4,ritm
...,...
727,salep
728,saksafon
729,ağabey
730,samyeli


In [39]:
df_diff_merge = pd.merge(df_turkish_english_all,df_diff, how="right", on="dict_entry_main")
df_diff_merge.drop_duplicates(inplace=True)
df_diff_merge

,dict_entry_main,english_word
0,arayüz,interface
1,abece,gentlemanly
2,gey,gay
3,tulumba,pump
4,ritm,rhythm
...,...,...
763,salep,salep
764,saksafon,saxophone
765,ağabey,brother
766,samyeli,samiel


In [41]:
df_diff_merge["dict_entry_main"].nunique()

732

In [40]:
df_diff_merge.to_excel("Turkish English Shared Difference Word.xlsx", index=False)

In [26]:
df_all_shared_concat = pd.concat([df_turkish_english_all,df_turkish_english_shared], axis=0)
df_all_shared_concat.drop_duplicates(inplace=True)
df_all_shared_concat

,dict_entry_main,english_word
0,sirk,circus
1,oksijen,oxygen
2,kaza,accident
3,sinema,cinema
4,sezon,season
...,...,...
2132,zodyak,zodiac
2133,zombi,zombie
2134,zooloji,zoology
2135,zoolojik,zoological


In [34]:
df_dublicated = df_all_shared_concat[df_all_shared_concat["dict_entry_main"].duplicated()].sort_values(by="dict_entry_main")
df_dublicated

,dict_entry_main,english_word
1053,aba,kebaya
3,abdest,ablution
8,ablatif,ablative
17,acente,agent
18,adaptasyon,adaption
...,...,...
451,çince,Chinese
452,çinli,Chinese
2084,ütopya,Utopia
1797,şeker,sugar


In [42]:
df_turkish_english_all2 = df_turkish_english_all.loc[:,["dict_entry_main"]]
df_turkish_english_shared2 = df_turkish_english_shared.loc[:,["dict_entry_main"]]

In [43]:
df_all_shared_concat2 = pd.concat([df_turkish_english_all2,df_turkish_english_shared2], axis=0)
df_all_shared_concat2.drop_duplicates(inplace=True)
df_all_shared_concat2

,dict_entry_main
0,sirk
1,oksijen
2,kaza
3,sinema
4,sezon
...,...
2132,zodyak
2133,zombi
2134,zooloji
2135,zoolojik


### Github Data Word Analysis 

In [50]:
df_turkish_english_shared = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_English_Shared_Vocabulary.xlsx")
df_turkish_english_shared.rename(columns={"english_word":"term"}, inplace=True)
df_turkish_english_shared = df_turkish_english_shared.loc[:,["term"]]
df_turkish_english_shared

,term
0,abacus
1,abandon
2,ebony
3,ablution
4,ablution room
...,...
2132,zodiac
2133,zombie
2134,zoology
2135,zoological


In [45]:
df_github_cross = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_cross_check2.csv")
df_github_cross

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,inherited_from,9w90qvZDUH-2OHPVZEGJSQ,Middle english,parlai
1,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,Uy6v5wWuW6KGH4aTqGWnmA,Old french,parler
2,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,HY24XyCcUZCYSa1Q6y2kPA,Late latin,parabolō
3,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,uXGtk5g-W8G_TlTxSHImpg,Latin,parabola
4,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,JHVmHqAPXEq-YaIbd0B5Jg,Ancient greek,παραβολή
...,...,...,...,...,...,...,...
8854119,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,inherited_from,TUnsO83vXXC1Us5rJgGyUg,Vulgar latin,*pittitus
8854120,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,oUnwLLsHVymmwwiRXgrRow,Latin,pitinnus
8854121,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,IGRZaHQ-VU-vpxMjdxSTlA,Latin,pitulus
8854122,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,MDRrWciZWXGEj1KR9MUWtg,Latin,piccinus


In [47]:
df_github_english = df_github_cross[df_github_cross["lang"] == "English"]
df_github_english

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
8,9dklyzlWU1GJG2HQTdn2Gw,English,breaker,borrowed_from,5hrBfLYUVKCz_apIVUIeCQ,Occitan,barrique
9,9dklyzlWU1GJG2HQTdn2Gw,English,breaker,derived_from,7k2Vvp3nXLy5CvsTUAGhnA,Old occitan,barrica
10,Sa9PLd8wVye-T_9yUc0DOw,English,unflappable,inherited_from,1s_U6-KHWmiJeKVbA96FWg,Middle english,un-
11,9dklyzlWU1GJG2HQTdn2Gw,English,breaker,derived_from,UJle1K0PXdyx7GycKtbdvQ,Vulgar latin,*barrica
12,Sa9PLd8wVye-T_9yUc0DOw,English,unflappable,inherited_from,tiwBatmsX3aT6yV_bBj4pA,Old english,un-
...,...,...,...,...,...,...,...
8854101,X9TbD7nbViavII1VL_kMCw,English,geotechnique,borrowed_from,l-rKJaTcXL65SH14LLHRgA,French,technique
8854102,X9TbD7nbViavII1VL_kMCw,English,geotechnique,etymologically_related_to,l-rKJaTcXL65SH14LLHRgA,French,technique
8854103,X9TbD7nbViavII1VL_kMCw,English,geotechnique,derived_from,YbyR0LCGX4qfUGCtxMAlNQ,Ancient greek,τεχνικός
8854104,X9TbD7nbViavII1VL_kMCw,English,geotechnique,etymologically_related_to,4bixcB-SUnyHLzonp8WarQ,Ancient greek,τέχνη


In [51]:
df_english_merge = pd.merge(df_github_english,df_turkish_english_shared, how="inner", on="term")
df_english_merge.drop_duplicates(inplace=True)
df_english_merge

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,1IGPY62nW8KN1nioERiZFA,English,cricket,inherited_from,s-11vcZjUh6MY4LMXN6CPg,Late latin,-ittus
1,1IGPY62nW8KN1nioERiZFA,English,cricket,inherited_from,_EQDgeTuU3m8M2afVLH2IA,Latin,-ātus
2,1IGPY62nW8KN1nioERiZFA,English,cricket,derived_from,BKXcfZZTU1OSENC_7Oq1QA,Middle dutch,kricken
3,1IGPY62nW8KN1nioERiZFA,English,cricket,etymologically_related_to,6ksGYWiZWTWhDUn3wI70Cg,Middle dutch,crieken
4,1IGPY62nW8KN1nioERiZFA,English,cricket,derived_from,KUro1PmgXzSqQ8FcWutqIQ,Proto-west germanic,*krakōn
...,...,...,...,...,...,...,...
27048,BJLslSsOVmKw6XHuuR2AMQ,English,caliber,borrowed_from,FluA-iVcUFKjzGVmRATqOw,Arabic,قَالِب
27049,BJLslSsOVmKw6XHuuR2AMQ,English,caliber,derived_from,FluA-iVcUFKjzGVmRATqOw,Arabic,قَالِب
27050,BJLslSsOVmKw6XHuuR2AMQ,English,caliber,derived_from,GisFPTZ8URiYXeEjr1tmBQ,Ancient greek,καλαπούς
27051,BJLslSsOVmKw6XHuuR2AMQ,English,caliber,derived_from,rEJNH8d0XCmAhwEg_YD02Q,Latin,qua


In [52]:
df_english_merge.term.nunique()

1726

#### Turkish English Shared Word Dutch German Portuguese Spanish Analysis 

In [5]:
lang_1 = "Turkish"  # English, Turkish, German, French, Spanish, Arabic, Russian, Portuguese, Dutch, Italian 
lang_2 = "English"  # English, Turkish, German, French, Spanish, Arabic, Russian, Portuguese, Dutch, Italian

if lang_2 == "English":
    lang_3 = "English (eng)"
elif lang_2 == "Turkish":
    lang_3 = "Turkish (tur)"
elif lang_2 == "German":
    lang_3 = "German (deu)"
elif lang_2 == "French":
    lang_3 = "French (fra)"
elif lang_2 == "Spanish":
    lang_3 = "Spanish (spa)"
elif lang_2 == "Arabic":
    lang_3 = "Arabic (ara)"
elif lang_2 == "Russian":
    lang_3 = "Russian (rus)"
elif lang_2 == "Portuguese":
    lang_3 = "Portuguese (por)"
elif lang_2 == "Dutch":
    lang_3 = "Dutch (nld)"
elif lang_2 == "Italian":
    lang_3 = "Italian (ita)"
else:
    pass

print(f"{lang_1}\n{lang_2}\n{lang_3}")

Turkish
English
English (eng)


In [8]:
df_github_cross = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_cross_check2.csv")
df_github_cross

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,inherited_from,9w90qvZDUH-2OHPVZEGJSQ,Middle english,parlai
1,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,Uy6v5wWuW6KGH4aTqGWnmA,Old french,parler
2,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,HY24XyCcUZCYSa1Q6y2kPA,Late latin,parabolō
3,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,uXGtk5g-W8G_TlTxSHImpg,Latin,parabola
4,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,JHVmHqAPXEq-YaIbd0B5Jg,Ancient greek,παραβολή
...,...,...,...,...,...,...,...
8854119,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,inherited_from,TUnsO83vXXC1Us5rJgGyUg,Vulgar latin,*pittitus
8854120,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,oUnwLLsHVymmwwiRXgrRow,Latin,pitinnus
8854121,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,IGRZaHQ-VU-vpxMjdxSTlA,Latin,pitulus
8854122,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,MDRrWciZWXGEj1KR9MUWtg,Latin,piccinus


In [6]:
df_turkish_english_shared = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_English_Shared_Vocabulary.xlsx")
df_turkish_english_shared.rename(columns={"english_word":"related_term"}, inplace=True)
df_turkish_english_shared = df_turkish_english_shared.loc[:,["related_term"]]
df_turkish_english_shared

,related_term
0,abacus
1,abandon
2,ebony
3,ablution
4,ablution room
...,...
2132,zodiac
2133,zombie
2134,zoology
2135,zoological


In [13]:
df_github_dutch_english = df_github_cross[(df_github_cross["lang"] == "Dutch") & (df_github_cross["related_lang"] == "English")]
df_github_dutch_english

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
1526,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,cognate_of,cjpWh1Y8XR-zFlyB9GS5mA,English,beer
1528,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,etymologically_related_to,cjpWh1Y8XR-zFlyB9GS5mA,English,beer
3818,f-wfvErPUJmzOOSRXqXZDQ,Dutch,woordenboek,cognate_of,ZN-AOGrDUmOM2SXV3SOA0Q,English,book
4170,f-wfvErPUJmzOOSRXqXZDQ,Dutch,woordenboek,cognate_of,81Q0H906Wpajp0il1_cjoQ,English,wordbook
4472,f-wfvErPUJmzOOSRXqXZDQ,Dutch,woordenboek,compound_of,4rWHed4DUf2hF3UCuYkwdQ,English,word
...,...,...,...,...,...,...,...
8852336,uS6r0FPTWXqdEH11d6ugdw,Dutch,goederenwagon,borrowed_from,PNhPpeX5W4KwtQTEmfG2DA,English,waggon
8853463,4XxPn-PdU4iyc9oLmtzxPA,Dutch,bruggenhoofd,semantic_loan_of,11FbUW5MWBG96joRM5lvJg,English,head
8853884,d4Wq0qZqUB6BZw9FLso8uQ,Dutch,Paarlo,etymologically_related_to,95aeMwhwV4CSs0TgBWeRJA,English,lere
8853918,J1LKhKHdXsytZjHyd3wDrQ,Dutch,Rothenbach,cognate_of,ccg4hiVMVQyWYkVNQjdyGg,English,red


In [98]:
df_github_dutch_english_merge = pd.merge(df_github_dutch_english,df_turkish_english_shared, how="inner", on="related_term")
df_github_dutch_english_merge.drop_duplicates(inplace=True)
df_github_dutch_english_merge 

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,cognate_of,cjpWh1Y8XR-zFlyB9GS5mA,English,beer
1,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,etymologically_related_to,cjpWh1Y8XR-zFlyB9GS5mA,English,beer
2,wio74nlAV5SuS25zmxHpwA,Dutch,lager,cognate_of,cjpWh1Y8XR-zFlyB9GS5mA,English,beer
3,wio74nlAV5SuS25zmxHpwA,Dutch,lager,etymologically_related_to,cjpWh1Y8XR-zFlyB9GS5mA,English,beer
4,XxzPhYdtXX-OF9Tc1C6ivQ,Dutch,cricket,is_onomatopoeic,1IGPY62nW8KN1nioERiZFA,English,cricket
...,...,...,...,...,...,...,...
1881,ha3ZJRXuV46lCDJgYXAGGg,Dutch,internetforum,derived_from,WjPQzXKtVfObfT6KK6p9rQ,English,forum
1882,ENfZ3QDbXwmi2aihs7qubg,Dutch,taboewoord,borrowed_from,4DFL3mncVnuGxtkdCW35Gw,English,taboo
1883,lrCrs9hFXhetFxs30QKdDg,Dutch,pijpkruid,compound_of,a2epdeLZURS8ZtswzQVnUw,English,pipe
1884,ohdheswTWu2w1pavUnoL6g,Dutch,ijscrème,compound_of,YaduaIt4WCGmgGb1je0Ucw,English,cream


In [99]:
df_github_dutch_english_merge =  df_github_dutch_english_merge.loc[:,["term","related_term"]]
df_github_dutch_english_merge.rename(columns={"term":"dutch_term","related_term":"english_term"}, inplace=True)
df_github_dutch_english_merge.drop_duplicates(inplace=True)
df_github_dutch_english_merge.reset_index(drop=True, inplace=True)
df_github_dutch_english_merge

,dutch_term,english_term
0,pils,beer
1,lager,beer
2,cricket,cricket
3,krekel,cricket
4,cricketen,cricket
...,...,...
1778,internetforum,forum
1779,taboewoord,taboo
1780,pijpkruid,pipe
1781,ijscrème,cream


In [100]:
df_github_german_english = df_github_cross[(df_github_cross["lang"] == "German") & (df_github_cross["related_lang"] == "English")]
df_github_german_english

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
2203,RdgMzboDVciQzRJdkxLQiQ,German,Master,doublet_with,CEP3D4yKU7WabIqN4jJ8Pw,English,maestro
2205,RdgMzboDVciQzRJdkxLQiQ,German,Master,doublet_with,rD7H3C-sUoyAlFFmvryl-Q,English,magister
2207,RdgMzboDVciQzRJdkxLQiQ,German,Master,has_prefix_with_root,ki2Ae3XoURSXSGfoEeRa7Q,English,mast
2209,RdgMzboDVciQzRJdkxLQiQ,German,Master,has_suffix,Yf91w-qWUGCrz1yPUqk1fg,English,er
2269,RdgMzboDVciQzRJdkxLQiQ,German,Master,cognate_of,C9oBjBAFUm6dh-dq5tm1jg,English,master
...,...,...,...,...,...,...,...
8853826,W0QtyHp_VjmnCJZKttP0HA,German,Totenfest,cognate_of,fXfgHaFFUGSQFRco3Ttcdg,English,feast
8854008,wWLyZfSdV3OUtte6YLRBzw,German,Nachtsicht,cognate_of,sYIDrsx2Vhe-Po1Wr8OhuA,English,night
8854108,r_86_CzTUmiwftlGf3Raow,German,Vordertür,cognate_of,jGY3ZCyuVyGGU0qlMo-p0Q,English,further
8854109,r_86_CzTUmiwftlGf3Raow,German,Vordertür,etymologically_related_to,THTyAO_PU3yS0g2dZJ3FhA,English,farther


In [101]:
df_github_german_english_merge = pd.merge(df_github_german_english,df_turkish_english_shared, how="inner", on="related_term")
df_github_german_english_merge.drop_duplicates(inplace=True)
df_github_german_english_merge 

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,RdgMzboDVciQzRJdkxLQiQ,German,Master,doublet_with,CEP3D4yKU7WabIqN4jJ8Pw,English,maestro
1,ScYJfDSeXBupdbaNwdZoWQ,German,Meister,doublet_with,CEP3D4yKU7WabIqN4jJ8Pw,English,maestro
2,RdgMzboDVciQzRJdkxLQiQ,German,Master,cognate_of,C9oBjBAFUm6dh-dq5tm1jg,English,master
3,J3rTwK5sVTqU28h9eP7aBA,German,Meisterschaft,cognate_of,C9oBjBAFUm6dh-dq5tm1jg,English,master
4,VuCTNeqrXqWbjs5wXy5Njg,German,Bademeister,cognate_of,C9oBjBAFUm6dh-dq5tm1jg,English,master
...,...,...,...,...,...,...,...
1690,0ZQp9skWXVeu8o1oKz-9gg,German,Generaladmiral,cognate_of,Y2EHbEqDV1KpL4RfjNEUmw,English,admiral
1691,rmM-wdGYVJOa8_jH57oJxg,German,Kommodore,etymologically_related_to,_AW2SPVZU9qt1kY1nnmMww,English,mandate
1692,LrApqhDgVB23kQPO88jCBg,German,Dollarzeichen,derived_from,TCyUChmtUduCh9z-qEN_6g,English,dollar
1693,Ik_kX-K4W0SjSA4BnIT-YA,German,Comedian,has_prefix_with_root,xU5Ui0wZX8ia8N1O50rXuA,English,comedy


In [102]:
df_github_german_english_merge =  df_github_german_english_merge.loc[:,["term","related_term"]]
df_github_german_english_merge.rename(columns={"term":"german_term","related_term":"english_term"}, inplace=True)
df_github_german_english_merge.drop_duplicates(inplace=True)
df_github_german_english_merge.reset_index(drop=True, inplace=True)
df_github_german_english_merge

,german_term,english_term
0,Master,maestro
1,Meister,maestro
2,Master,master
3,Meisterschaft,master
4,Bademeister,master
...,...,...
1592,Generaladmiral,admiral
1593,Kommodore,mandate
1594,Dollarzeichen,dollar
1595,Comedian,comedy


In [103]:
df_github_portuguese_english = df_github_cross[(df_github_cross["lang"] == "Portuguese") & (df_github_cross["related_lang"] == "English")]
df_github_portuguese_english

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
985,swzwZAVGWQiDC5rPCf8Ubw,Portuguese,net,borrowed_from,B7-Pfg61VXS_CaWed37sDg,English,internet
3773,pNuJBWoAWwWSCv2SRvJ5Dg,Portuguese,admirador,etymologically_related_to,vcMIWROgVY2LW1_ku6AWKA,English,admire
5283,gDmC1nRsX6WIrzmW8KQGag,Portuguese,bestseller,compound_of,HpX5T6ELUxCaxMHVxzVJsA,English,best
5284,gDmC1nRsX6WIrzmW8KQGag,Portuguese,bestseller,compound_of,fGk4e5HNX62dKk4_CtWEww,English,seller
8449,BhYXitGEVhiJH6CTd7vZJg,Portuguese,antiquark,borrowed_from,tmJh6-HXXlStAk01QXlH5w,English,quark
...,...,...,...,...,...,...,...
8847018,zgVYL7YtWLyNdiaa8hPg-w,Portuguese,joguete,borrowed_from,iZkSmNZ6Ws6g6GPa5UwyGA,English,-ette
8848005,LY46SXPmXsiYX2OSBJa9bQ,Portuguese,resetar,borrowed_from,qwzLY779W_iX5pbmmHuBJg,English,reset
8848029,Y02c9L9VWy2v2L0amLXlwQ,Portuguese,Fordlandia,has_affix,Azf7bnMZXTSmJp9IShKo5A,English,Ford
8848030,Y02c9L9VWy2v2L0amLXlwQ,Portuguese,Fordlandia,has_affix,6PSERXX0VXqKQkIFCzSQgw,English,land


In [104]:
df_github_portuguese_english_merge = pd.merge(df_github_portuguese_english,df_turkish_english_shared, how="inner", on="related_term")
df_github_portuguese_english_merge.drop_duplicates(inplace=True)
df_github_portuguese_english_merge 

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,qPCkp86bU5-VRg6vcxQfHA,Portuguese,super,etymologically_related_to,mrdMJzquWsigQdN8iEiQUQ,English,super
1,QB3I_kaYWH2_HvYmi8F3AA,Portuguese,sobre,etymologically_related_to,mrdMJzquWsigQdN8iEiQUQ,English,super
2,FaXiwtkDU7mI6m3EyRgexA,Portuguese,sobressair,etymologically_related_to,mrdMJzquWsigQdN8iEiQUQ,English,super
3,sZjYD5sqUfqjeEkVxyPARg,Portuguese,supermercado,etymologically_related_to,mrdMJzquWsigQdN8iEiQUQ,English,super
4,U75Yll_GVnylTjltQQ6-xQ,Portuguese,supervisor,etymologically_related_to,mrdMJzquWsigQdN8iEiQUQ,English,super
...,...,...,...,...,...,...,...
421,eu48vXiUWLifMAtkbBYw6Q,Portuguese,autocompletar,has_prefix_with_root,V-91sQZzWa6RyLY4ATPF-Q,English,complete
422,3T6rozhcVHmo4JgRCmqj_g,Portuguese,broderagem,borrowed_from,-E-t7V8FXQ2YIB-TvvB7Lw,English,brother
423,LZvgVudcVgCfzIriOE_jnw,Portuguese,videoárbitro,borrowed_from,lsfZdOriUemEYER4HmaW_A,English,video
424,5uUu8D9cU6y108rIsMwbGQ,Portuguese,aidético,borrowed_from,JbfegdsXWBGCJkUtj3xkwQ,English,AIDS


In [105]:
df_github_portuguese_english_merge =  df_github_portuguese_english_merge.loc[:,["term","related_term"]]
df_github_portuguese_english_merge.rename(columns={"term":"portuguese_term","related_term":"english_term"}, inplace=True)
df_github_portuguese_english_merge.drop_duplicates(inplace=True)
df_github_portuguese_english_merge.reset_index(drop=True, inplace=True)
df_github_portuguese_english_merge

,portuguese_term,english_term
0,super,super
1,sobre,super
2,sobressair,super
3,supermercado,super
4,supervisor,super
...,...,...
391,autocompletar,complete
392,broderagem,brother
393,videoárbitro,video
394,aidético,AIDS


In [106]:
df_github_spanish_english = df_github_cross[(df_github_cross["lang"] == "Spanish") & (df_github_cross["related_lang"] == "English")]
df_github_spanish_english

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
7,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,etymologically_related_to,r3NclALnW1ioq3gnbe1R5g,English,palaver
137,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,doublet_with,fS3-uXGaUqmu9O5GHb3cLg,English,parable
138,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,doublet_with,WB7OVE01XeSxWXeHsxGrxQ,English,parole
139,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,doublet_with,vUZK0nDZUx6iL1yjxXEGYA,English,parabola
268,h8QhUhlEWRGV1MKt1nkccA,Spanish,nosotros,etymologically_related_to,liR54wB8UoyXcIBemBiaZA,English,other
...,...,...,...,...,...,...,...
8837321,DoQJzIthXgi0AjDN7EY6ww,Spanish,mascné,blend_of,WudOE-ODWfO3g5cx7SR0AA,English,acne
8842680,OoAMt_uxWWu8eVpvpjT9Cw,Spanish,fotingo,etymologically_related_to,03bZUQSRWb6OjrCDa_iigw,English,yode
8842682,OoAMt_uxWWu8eVpvpjT9Cw,Spanish,fotingo,etymologically_related_to,5qedyU5bVr68S2mWn39jKw,English,went
8847410,57RKS5pVWpiNkXXn0MlBkw,Spanish,me vale madre,cognate_of,xdj6HtRLWPGOHyqi1dWu4A,English,matron


In [107]:
df_github_spanish_english_merge = pd.merge(df_github_spanish_english,df_turkish_english_shared, how="inner", on="related_term")
df_github_spanish_english_merge.drop_duplicates(inplace=True)
df_github_spanish_english_merge 

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,doublet_with,vUZK0nDZUx6iL1yjxXEGYA,English,parabola
1,wOfbXZCWXJyr-_3aM9YdXQ,Spanish,parole,doublet_with,vUZK0nDZUx6iL1yjxXEGYA,English,parabola
2,Pck6ku8NWFmYGEuKyfA3lw,Spanish,parlotear,doublet_with,vUZK0nDZUx6iL1yjxXEGYA,English,parabola
3,HQZL1KfJW8e_n0kHIw_MiQ,Spanish,alérgico,etymologically_related_to,1wCvICq9UWWYaiETZRJlJg,English,allergy
4,QjeguBtuXiOdIPLhf88bdQ,Spanish,alérgeno,etymologically_related_to,1wCvICq9UWWYaiETZRJlJg,English,allergy
...,...,...,...,...,...,...,...
2323,-WGxZYkVWJu3F1GZgdVoaQ,Spanish,buchaca,cognate_of,bL1zSIB_XMiWM3uAp3OYgw,English,budget
2324,E35bAyzbWNiWpJ-j3esVSg,Spanish,exdetective,borrowed_from,A2ScUk3-X1yMa4MVc3Px3Q,English,detective
2325,xioi3RBkVCSfsttbl4VSuQ,Spanish,búrguer,clipping_of,Wv3wvtt5UXuh40wXLtANAA,English,hamburger
2326,DoQJzIthXgi0AjDN7EY6ww,Spanish,mascné,blend_of,Xm-4mnzxXMW-4Vgjc1rRjw,English,mask


In [108]:
df_github_spanish_english_merge =  df_github_spanish_english_merge.loc[:,["term","related_term"]]
df_github_spanish_english_merge.rename(columns={"term":"spanish_term","related_term":"english_term"}, inplace=True)
df_github_spanish_english_merge.drop_duplicates(inplace=True)
df_github_spanish_english_merge.reset_index(drop=True, inplace=True)
df_github_spanish_english_merge

,spanish_term,english_term
0,palabra,parabola
1,parole,parabola
2,parlotear,parabola
3,alérgico,allergy
4,alérgeno,allergy
...,...,...
2241,buchaca,budget
2242,exdetective,detective
2243,búrguer,hamburger
2244,mascné,mask


In [109]:
df_turkish_english_shared = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_English_Shared_Vocabulary.xlsx")
df_turkish_english_shared.rename(columns={"english_word":"english_term"}, inplace=True)
df_turkish_english_shared = df_turkish_english_shared.loc[:,["english_term"]]
df_turkish_english_shared

,english_term
0,abacus
1,abandon
2,ebony
3,ablution
4,ablution room
...,...
2132,zodiac
2133,zombie
2134,zoology
2135,zoological


In [110]:
df1 = pd.merge(df_turkish_english_shared,df_github_dutch_english_merge, how="left", on="english_term")
df1.drop_duplicates(inplace=True)
df1.reset_index(drop=True, inplace=True)
df1

,english_term,dutch_term
0,abacus,NaN
1,abandon,ban
2,abandon,bannen
3,ebony,NaN
4,ablution,NaN
...,...,...
3493,zoom,Zoom
3494,zoom,inzoomen
3495,zoom,uitzoomen
3496,zoom,zomen


In [111]:
df2 = pd.merge(df1,df_github_german_english_merge, how="left", on="english_term")
df2.drop_duplicates(inplace=True)
df2.reset_index(drop=True, inplace=True)
df2

,english_term,dutch_term,german_term
0,abacus,NaN,NaN
1,abandon,ban,NaN
2,abandon,bannen,NaN
3,ebony,NaN,NaN
4,ablution,NaN,NaN
...,...,...,...
60331,zoom,zomen,Zoomobjektiv
60332,zoom,schildzoom,Saum
60333,zoom,schildzoom,herauszoomen
60334,zoom,schildzoom,hineinzoomen


In [112]:
df3 = pd.merge(df2,df_github_portuguese_english_merge, how="left", on="english_term")
df3.drop_duplicates(inplace=True)
df3.reset_index(drop=True, inplace=True)
df3

,english_term,dutch_term,german_term,portuguese_term
0,abacus,NaN,NaN,NaN
1,abandon,ban,NaN,NaN
2,abandon,bannen,NaN,NaN
3,ebony,NaN,NaN,NaN
4,ablution,NaN,NaN,NaN
...,...,...,...,...
132368,zoom,zomen,Zoomobjektiv,NaN
132369,zoom,schildzoom,Saum,NaN
132370,zoom,schildzoom,herauszoomen,NaN
132371,zoom,schildzoom,hineinzoomen,NaN


In [113]:
df4 = pd.merge(df3,df_github_spanish_english_merge, how="left", on="english_term")
df4.drop_duplicates(inplace=True)
df4.reset_index(drop=True, inplace=True)
df4

,english_term,dutch_term,german_term,portuguese_term,spanish_term
0,abacus,NaN,NaN,NaN,NaN
1,abandon,ban,NaN,NaN,bando
2,abandon,ban,NaN,NaN,banear
3,abandon,bannen,NaN,NaN,bando
4,abandon,bannen,NaN,NaN,banear
...,...,...,...,...,...
395690,zoom,zomen,Zoomobjektiv,NaN,NaN
395691,zoom,schildzoom,Saum,NaN,NaN
395692,zoom,schildzoom,herauszoomen,NaN,NaN
395693,zoom,schildzoom,hineinzoomen,NaN,NaN


In [118]:
df4.to_excel("Turkish English Shared Word Dutch German Portuguese Spanish Analysis.xlsx", index=False)

In [114]:
df4.dutch_term.nunique()

1727

In [115]:
df4.german_term.nunique()

1541

In [116]:
df4.portuguese_term.nunique()

364

In [117]:
df4.spanish_term.nunique()

1163

In [14]:
df_dutch = df_github_cross[df_github_cross.lang == "Dutch"]
df_dutch

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
1369,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,clipping_of,pmWg9nHdXkKgOGmsjGnT8w,German,Pilsener
1370,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,etymologically_related_to,RRFFFKyqWlm4FCP3v3t_aQ,German,Bier
1371,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,etymologically_related_to,UJtoeJ7-V8mBED0YCGXhiA,German,Pilsen
1372,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,etymologically_related_to,f2HO7kydVT2-uJaeNigWyg,Czech,Plzeň
1443,D3GcYQukU9m2ysG-lQAnQw,Dutch,pils,has_affix,UJtoeJ7-V8mBED0YCGXhiA,German,Pilsen
...,...,...,...,...,...,...,...
8854031,hizIVwJWWOqbH42n1yJCnQ,Dutch,Aan de Berg,inherited_from,dT27eZmEV9Sp7acanwBvVA,Middle dutch,berch
8854032,hizIVwJWWOqbH42n1yJCnQ,Dutch,Aan de Berg,inherited_from,CqlhJ_TRXNCtZst-zyTbdg,Old dutch,berg
8854033,hizIVwJWWOqbH42n1yJCnQ,Dutch,Aan de Berg,inherited_from,U_l-3UJXVsune11UJv4NRA,Proto-west germanic,*berg
8854034,hizIVwJWWOqbH42n1yJCnQ,Dutch,Aan de Berg,inherited_from,Y_1cZjZKVC-CEk_7QNGSYg,Proto-germanic,*bergaz


In [15]:
df_dutch[df_dutch.term == "agent"]

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
500061,qPnpXraKVuS9DPvwXXYROA,Dutch,agent,inherited_from,k4czcLKEV9CD61JsCQ1ezw,Middle french,agent
500062,qPnpXraKVuS9DPvwXXYROA,Dutch,agent,derived_from,EaUQjslTUCSv-o-ZCZfGVQ,Latin,agens


In [16]:
df_english = df_github_cross[df_github_cross.lang == "English"]
df_english

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
8,9dklyzlWU1GJG2HQTdn2Gw,English,breaker,borrowed_from,5hrBfLYUVKCz_apIVUIeCQ,Occitan,barrique
9,9dklyzlWU1GJG2HQTdn2Gw,English,breaker,derived_from,7k2Vvp3nXLy5CvsTUAGhnA,Old occitan,barrica
10,Sa9PLd8wVye-T_9yUc0DOw,English,unflappable,inherited_from,1s_U6-KHWmiJeKVbA96FWg,Middle english,un-
11,9dklyzlWU1GJG2HQTdn2Gw,English,breaker,derived_from,UJle1K0PXdyx7GycKtbdvQ,Vulgar latin,*barrica
12,Sa9PLd8wVye-T_9yUc0DOw,English,unflappable,inherited_from,tiwBatmsX3aT6yV_bBj4pA,Old english,un-
...,...,...,...,...,...,...,...
8854101,X9TbD7nbViavII1VL_kMCw,English,geotechnique,borrowed_from,l-rKJaTcXL65SH14LLHRgA,French,technique
8854102,X9TbD7nbViavII1VL_kMCw,English,geotechnique,etymologically_related_to,l-rKJaTcXL65SH14LLHRgA,French,technique
8854103,X9TbD7nbViavII1VL_kMCw,English,geotechnique,derived_from,YbyR0LCGX4qfUGCtxMAlNQ,Ancient greek,τεχνικός
8854104,X9TbD7nbViavII1VL_kMCw,English,geotechnique,etymologically_related_to,4bixcB-SUnyHLzonp8WarQ,Ancient greek,τέχνη


In [17]:
df_english[df_english.term == "agent"]

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
499743,G3OlWvrLUuao59Yjdn6FiA,English,agent,has_root,9soDv1a5Uwmf4MZ_LhhQXQ,Proto-indo-european,*h₂eǵ-
499744,G3OlWvrLUuao59Yjdn6FiA,English,agent,inherited_from,McT480c3VCyOvazYCfcTXg,Proto-italic,*agō
499745,G3OlWvrLUuao59Yjdn6FiA,English,agent,inherited_from,mkKF3eBYUxiYipN5u1dwHg,Proto-indo-european,*h₂éǵeti
499746,G3OlWvrLUuao59Yjdn6FiA,English,agent,cognate_of,kaBT8LTWUCSYMOwoioeu_w,Old irish,aigid
499747,G3OlWvrLUuao59Yjdn6FiA,English,agent,cognate_of,HoGcohhQUGeUXT-_a3zF1g,Ancient greek,ἄγω
499748,G3OlWvrLUuao59Yjdn6FiA,English,agent,cognate_of,J54dGMBbWNSb-qAljdDw7g,Old norse,aka
499749,G3OlWvrLUuao59Yjdn6FiA,English,agent,cognate_of,T1TGnrz7Xe26erKSYvTBhQ,Avestan,𐬀𐬰𐬀𐬌𐬙𐬌
499750,G3OlWvrLUuao59Yjdn6FiA,English,agent,cognate_of,mwKS3d7PX_2rWNtbAafvEQ,Sanskrit,अजति


#### Nisanyan Analysis

In [3]:
df_nisan = pd.read_json("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/nisanyan.json").transpose()
df_nisan

,baslik,tarihce,koken,daha_fazla,ek_aciklama,benzer_sozcukler,maddeye_gonderenler,tarih
ab,ab,"<p>[ Codex Cumanicus, 1303]<br/><span style=""d...",<p> <b>Farsça</b> <i>āb </i>veya<i> āv</i> آ...,,<p>Aynı İrani kökten <b>Avesta (Zend) dilinde<...,"[abıhayat, abıkevser]","[abdest, avadanlık, cillop, çirkef, dolap (tül...",29.10.2020
aba,aba,"<p>[ anon., Tezkiretü'l-Evliya terc., 1341]<b...",<p> <b>Arapça</b> <i>abāˀ </i>veya<i> ḳabāˀ</...,[kaban],,"[abacı, abalı]",[aban-],31.01.2018
abajur,abajur,"<p>""alafranga pencere kafesi, panjur"" [ Vartan...","<p> <b>Fransızca\n</b> <i>abat-jour</i> ""«gü...","[bateri, jurnal]","<p>Karş. <a href=""?k=panjur&amp;lnk=1"">panjur....",,[panjur],19.08.2017
abaküs,abaküs,"<p>[ Özön, Türkçe-Yabancı Kelimeler Sözlüğü, ...","<p> <b>Latince</b> <i>abacus</i> ""1. her tür...",,,,,15.11.2013
aban-,aban|mak,"<p>[ Ahmed Vefik Paşa, Lehce-ı Osmani, 1876]<...","<p> <b>Arapça</b> <i>abāˀ</i> ""keçe üstlük"" ...",[aba],,[abandırmak],,19.07.2013
...,...,...,...,...,...,...,...,...
zünnar,zünnar,"<p>[ anon., Mukaddimetü'l-Edeb terc., y. 1300...",<p> <b>Arapça</b> <i>znr</i> kökünden gelen ...,[zon],,,,12.03.2018
züppe,züppe,"<p>[ Mehmed Bahaeddin (Toven), Yeni Türkçe Lu...",<p> Bu sözcüğün kökeni belirsizdir. </p>,,"<p><b>Yunanca</b> <i>zóppos</i> ζόππος ""geri z...","[züppeleşmek, züppelik]",,03.02.2018
zürafa,zürafa,"<p>[ Lugat-i Halimi, 1477]<br/><span style=""di...",<p> <b>Arapça</b> <i>zrf</i> kökünden gelen ...,,,,,12.06.2015
zürefa,zürefa,"<p>[ Ahmed b. Kadı-i Manyas, Gülistan tercüme...",<p> <b>Arapça</b> <i>ẓrf</i> kökünden gelen ...,[zarif],,[zemheri zürefası],,03.07.2021
